In [ ]:
import json
import os

import pandas as pd
from dotenv import load_dotenv

from src.api_utils import get_statement, process_statement_data, send_request
from src.transformations import (
    add_cum_sum_rows,
    get_filtered_data,
    get_filtered_pcr,
    transform,
)
from src.visualizations import create_initial_plot, generate_buttons

pd.options.mode.copy_on_write = True

In [ ]:
load_dotenv()
token = os.getenv("TOKEN")
if token is None:
    raise ValueError("Please set the TOKEN environment variable.")
query_id = os.getenv("QUERY_ID")
if query_id is None:
    raise ValueError("Please set the QUERY_ID environment variable.")
flex_version = os.getenv("FLEX_VERSION")
if flex_version is None:
    raise ValueError("Please set the FLEX_VERSION environment variable.")
headers_str = os.getenv("HEADERS")
if headers_str is None:
    raise ValueError("Please set the HEADERS environment variable.")
headers = json.loads(headers_str)

### Get Statement from IB via API

In [ ]:
# API Calls
reference_code = send_request(token, query_id, flex_version, headers)
df = get_statement(token, reference_code, flex_version, headers)

# Process Data
df = process_statement_data(df)

In [ ]:
df_transformed = transform(df)

In [ ]:
pd.set_option("display.max_rows", None)
df_transformed.head(500)

### Visualize Results

In [ ]:
# filtered_df is needed for PCR later
filtered_df, grouped_df = filter_and_group_data(
    df,
    assets_to_include=["OPT", "FOP"],
    symbols_to_include=None,
    dates_to_exclude=None,
)

In [ ]:
filtered_df.head(200)

In [ ]:
grouped_df = add_cum_sum_rows(grouped_df)
grouped_df

In [ ]:
buttons, buttons_asset, buttons_symbol = generate_buttons(grouped_df, get_filtered_data)

In [ ]:
fig = create_initial_plot(grouped_df, buttons, buttons_asset, buttons_symbol)
fig

### PCR

In [ ]:
result = get_filtered_pcr(filtered_df, "total")

In [ ]:
for w in sorted(grouped_df["week"].unique()):
    print(f"Woche {w}")
    result = get_filtered_pcr(filtered_df, "week", w)

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Beispiel-DataFrame (ersetzen Sie dies durch Ihre tatsächlichen Daten)
data = {
    "assetCategory": ["A", "A", "B", "B", "C", "C"],
    "underlyingSymbol": ["X", "Y", "X", "Z", "Y", "Z"],
    "value": [10, 15, 13, 17, 20, 12],
}
df = pd.DataFrame(data)


# Funktion zum Erstellen von Filterbuttons
def create_filter_buttons(df, column):
    buttons = []
    for value in df[column].unique():
        buttons.append(
            dict(label=value, method="update", args=[{"visible": df[column] == value}])
        )
    return buttons


# Erstellen von Buttons für assetCategory
asset_category_buttons = create_filter_buttons(df, "assetCategory")

# Erstellen von Buttons für underlyingSymbol
underlying_symbol_buttons = create_filter_buttons(df, "underlyingSymbol")

# Erstellen der Figur
fig = go.Figure(data=[go.Bar(x=df["assetCategory"], y=df["value"])])

# Hinzufügen der Updatemenüs zum Layout
fig.update_layout(
    updatemenus=[
        dict(
            buttons=asset_category_buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.2,
            yanchor="top",
        ),
        dict(
            buttons=underlying_symbol_buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.5,
            xanchor="left",
            y=1.2,
            yanchor="top",
        ),
    ]
)

# Layout-Anpassungen
fig.update_layout(title_text="Diagramm mit Filterbuttons", showlegend=False)

fig.show()
